In [6]:
import pandas as pd 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences 
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping 


# Read Dataset, Do Pre-Process

In [8]:
d = pd.read_csv('spam.csv', delimiter =',' , encoding = 'latin-1')

In [9]:
d.drop( ['Unnamed: 2' , 'Unnamed: 3' , 'Unnamed: 4'], axis=1 , inplace=True)

In [11]:
X = d.v2
Y = d.v1  
l = LabelEncoder() 
Y = l.fit_transform(Y) 
Y = Y.reshape(-1 , 1 ) 

In [12]:
X_train , X_test , Y_train, Y_test = train_test_split(X, Y, test_size = 0.15)

In [17]:
max_w = 1000
max_l = 150
tok = Tokenizer(num_words = max_w) 
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences , maxlen= max_l)

# Adding Layers

In [23]:
def RNN():
 inputs = Input(name = 'inputs', shape = [max_l])
 layer = Embedding(max_w, 50, input_length = max_l)(inputs)
 layer = LSTM(64)(layer)
 layer = Dense(256,name = 'FC1')(layer)
 layer = Activation('relu')(layer)
 layer = Dropout(0.5)(layer)
 layer = Dense(1,name = 'out_layer')(layer)
 layer = Activation('sigmoid')(layer)
 model = Model(inputs = inputs, outputs = layer)
 return model

# Create Model

In [24]:
model = RNN()

#Compile Model

In [26]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(),
metrics = ['accuracy'])


# Fit the Model

In [27]:
model.fit(  sequences_matrix,  Y_train,  batch_size = 128,  epochs=10,  validation_split = 0.2,
 callbacks=[EarlyStopping(monitor = 'val_loss', min_delta = 0.0001) ] )

Epoch 1/10
30/30 [==============================] - 9s 199ms/step - loss: 0.3130 - accuracy: 0.8902 - val_loss: 0.1252 - val_accuracy: 0.9652
Epoch 2/10
30/30 [==============================] - 6s 196ms/step - loss: 0.0837 - accuracy: 0.9778 - val_loss: 0.0429 - val_accuracy: 0.9884


# Save Model

In [28]:
model.save('./Spam.h5')

#Test Model

In [29]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen = max_l)
accr = model.evaluate(test_sequences_matrix, Y_test)
print('Test set\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(accr[0],accr[1]))


27/27 [==============================] - 1s 27ms/step - loss: 0.0488 - accuracy: 0.9880
Test set
 Loss: 0.049
 Accuracy: 0.988
